# Quarterly Roll Up 
* 6/9: Something is wrong with it, there are multiple rows repeated for the same quarter when there should only be one row for one quarter for one route. 
[Issue](https://github.com/cal-itp/data-analyses/issues/1416)

In [1]:
import _report_route_dir_visuals
import _route_dir_data_prep
import altair as alt
import calitp_data_analysis.magics
import deploy_portfolio_yaml
import pandas as pd
import quarterly_rollup
import yaml
from IPython.display import HTML, Image, Markdown, display, display_html
from segment_speed_utils import (
    gtfs_schedule_wrangling,
    metrics,
    project_vars,
    segment_calcs,
    time_series_utils,
)
from shared_utils import (
    catalog_utils,
    gtfs_utils_v2,
    portfolio_utils,
    publish_utils,
    rt_dates,
    rt_utils,
    time_helpers,
)
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
from omegaconf import OmegaConf

readable_dict = OmegaConf.load("readable2.yml")

In [4]:
analysis_date_list = rt_dates.y2025_dates

In [5]:
#  portfolio_name = "City and County of San Francisco"
portfolio_name = "Ventura County (VCTC, Gold Coast, Cities of Camarillo, Moorpark, Ojai, Simi Valley, Thousand Oaks)"

In [6]:
# Read in og df
DIGEST_RT_SCHED_MONTH = GTFS_DATA_DICT.digest_tables.monthly_route_schedule_vp

In [7]:
monthly_df = pd.read_parquet(f"{RT_SCHED_GCS}{DIGEST_RT_SCHED_MONTH}.parquet")

In [8]:
one_route = monthly_df.loc[
    (monthly_df.portfolio_organization_name == portfolio_name)
    & (monthly_df.recent_combined_name == "80-89 Coastal Express")
]

In [9]:
one_route[
    ["recent_route_id", "recent_combined_name", "combined_name"]
].drop_duplicates()

,recent_route_id,recent_combined_name,combined_name
222269,4134,80-89 Coastal Express,80__80-89 Coastal Express
223550,4136,80-89 Coastal Express,80X__80-89 Coastal Express
224432,4137,80-89 Coastal Express,81__80-89 Coastal Express
225314,4138,80-89 Coastal Express,81B__80-89 Coastal Express
226196,4141,80-89 Coastal Express,84__80-89 Coastal Express
226994,4142,80-89 Coastal Express,84U__80-89 Coastal Express
227876,4143,80-89 Coastal Express,85__80-89 Coastal Express
228674,4144,80-89 Coastal Express,85C__80-89 Coastal Express
229556,4145,80-89 Coastal Express,86__80-89 Coastal Express
230438,4146,80-89 Coastal Express,87__80-89 Coastal Express


In [11]:
DIGEST_RT_SCHED_QTR = "digest/quarterly_schedule_vp_metrics"

In [12]:
qtr_df_script = pd.read_parquet(f"{RT_SCHED_GCS}{DIGEST_RT_SCHED_QTR}.parquet")

In [15]:
one_route_qtr_df = qtr_df_script.loc[
    (qtr_df_script["portfolio_organization_name"] == portfolio_name)
    & (qtr_df_script["recent_combined_name"] == "80-89 Coastal Express")
]

In [18]:
one_route_qtr_df.loc[(one_route_qtr_df.time_period == "all_day") & (one_route_qtr_df.year_quarter == "2025 Q1")][["pct_rt_journey_atleast1_vp", "pct_rt_journey_atleast2_vp"]]

,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp
11561,0.51,0.50
11687,0.66,0.66
